# 📈 Tesla Stock Forecasting (2014–2023) Using ARIMA & SARIMA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# Load Tesla dataset
df = pd.read_csv('tsla_2014_2023.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)
df = df[['Close']]
df = df.asfreq('B')  # Business day frequency
df = df.fillna(method='ffill')
df.plot(title='Tesla Stock Price (2014–2023)', figsize=(10,5))
plt.ylabel('Closing Price ($)')
plt.show()

In [ ]:
# ADF test for stationarity
result = adfuller(df['Close'])
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
df_diff = df.diff().dropna()
df_diff.plot(title='Differenced Tesla Stock Prices')
plt.show()

In [ ]:
# ACF and PACF plots
plot_acf(df_diff)
plt.show()
plot_pacf(df_diff)
plt.show()

In [ ]:
# ARIMA model
model_arima = ARIMA(df['Close'], order=(1,1,1))
result_arima = model_arima.fit()
print(result_arima.summary())
result_arima.plot_predict(dynamic=False)
plt.show()

In [ ]:
# SARIMA model
model_sarima = SARIMAX(df['Close'], order=(1,1,1), seasonal_order=(1,1,1,252))
result_sarima = model_sarima.fit()
print(result_sarima.summary())
result_sarima.plot_diagnostics(figsize=(12,6))
plt.show()

In [ ]:
# Forecast next 30 days
forecast = result_sarima.get_forecast(steps=30)
forecast_ci = forecast.conf_int()
forecast.predicted_mean.plot(label='Forecast')
df['Close'].plot(label='Observed')
plt.fill_between(forecast_ci.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1],
                 color='lightblue', alpha=0.3)
plt.title('Tesla Stock Price Forecast (Next 30 Days)')
plt.legend()
plt.show()